# Lesson 1 solutions: Finding Higgs decays

<br><br><br><br><br>

## Getting data, building objects

In [1]:
import json

import numpy as np
import vector

dataset = json.load(open("../data/SMHiggsToZZTo4L.json"))

def to_vector(particle):
    return vector.obj(
        pt=particle["pt"],
        eta=particle["eta"],
        phi=particle["phi"],
        mass=particle["mass"],
    )

In [2]:
class Particle:
    def __init__(self, E, px, py, pz, charge, flavor):
        self.E = E
        self.px = px
        self.py = py
        self.pz = pz
        self.charge = charge
        self.flavor = flavor

    def __repr__(self):
        return f"<{type(self).__name__} E={self.E} px={self.px} py={self.py} pz={self.pz} charge={self.charge} flavor={self.flavor!r}>"

    # the '__add__' method gives meaning to 'particle + particle'
    def __add__(self, other):
        return Particle(
            self.E + other.E,
            self.px + other.px,
            self.py + other.py,
            self.pz + other.pz,
            self.charge + other.charge,
            "none"
            if self.charge + other.charge == 0
            else f"{self.flavor}-{other.flavor}",
        )

    # '@property' means we can call this method without parentheses, as though it were an attribute
    @property
    def mass(self):
        return vector.obj(E=self.E, px=self.px, py=self.py, pz=self.pz).mass

In [3]:
leptons = []  # a "lepton" is an electron or a muon, distinguished by its "flavor"

event = dataset[96]  # a nice event with 3 electrons and 3 muons

for particle in event["electron"]:
    v = to_vector(particle)
    leptons.append(Particle(v.E, v.px, v.py, v.pz, particle["charge"], "electron"))

for particle in event["muon"]:
    v = to_vector(particle)
    leptons.append(Particle(v.E, v.px, v.py, v.pz, particle["charge"], "muon"))

In [4]:
z_candidates_step0 = []
for i, lepton_i in enumerate(leptons):
    for j, lepton_j in enumerate(leptons):
        if i < j:
            z_candidates_step0.append({"index": [i, j], "Z boson": lepton_i + lepton_j})

<br><br><br><br><br>

## Exercise part 1

Z bosons always decay into particles of opposite charge and identical flavor. Reduce the set of candidates by excluding ones with the wrong properties.

In [5]:
z_candidates_step1 = []
for candidate in z_candidates_step0:
    i, j = candidate["index"]
    z_boson = candidate["Z boson"]
    if z_boson.charge == 0 and z_boson.flavor == "none":
        z_candidates_step1.append(candidate)

Print the masses of these Z boson candidates.

In [6]:
for candidate in z_candidates_step1:
    print(candidate["Z boson"].mass)

94.65200565609616
62.03397488944119
3.4170504361020386
3.087732909498452
45.69023328291948
3.6622583780140623
26.45024522236556
3.27373703909123


or use a Python list comprehension:

In [7]:
[candidate["Z boson"].mass for candidate in z_candidates_step1]

[94.65200565609616,
 62.03397488944119,
 3.4170504361020386,
 3.087732909498452,
 45.69023328291948,
 3.6622583780140623,
 26.45024522236556,
 3.27373703909123]

<br><br><br><br><br>

## Exercise part 2

The Higgs boson decays into two Z bosons. The only constraint here is that a lepton from one Z decay can't also be a lepton from the other Z decay.

In [8]:
higgs_candidates_step1 = []

for z_index1, z_candidate1 in enumerate(z_candidates_step1):
    for z_index2, z_candidate2 in enumerate(z_candidates_step1):
        if z_index1 < z_index2:
            lepton_i1, lepton_j1 = z_candidate1["index"]
            lepton_i2, lepton_j2 = z_candidate2["index"]
            z_boson1 = z_candidate1["Z boson"]
            z_boson2 = z_candidate2["Z boson"]

            if (lepton_i1 != lepton_i2 and lepton_i1 != lepton_j2) and (lepton_j1 != lepton_i2 and lepton_j1 != lepton_j2):
                higgs_candidates_step1.append(
                    {
                        "index": [lepton_i1, lepton_j1, lepton_i2, lepton_j2],
                        "H boson": z_boson1 + z_boson2,
                        "Z bosons": [z_boson1, z_boson2],
                    }
                )

You could have also used

```python
if lepton_i1 not in (lepton_i2, lepton_j2) and lepton_j1 not in (lepton_i2, lepton_j2):
```

or several other, equivalent variants.

Print out the indexes and masses of the candidates.

In [9]:
for higgs_candidate in higgs_candidates_step1:
    print(higgs_candidate["index"], higgs_candidate["H boson"].mass)

[0, 2, 1, 3] 118.25980390632363
[0, 2, 3, 4] 129.03461596915875
[0, 2, 3, 5] 118.83117770896321
[0, 3, 1, 2] 118.25980390632363
[0, 3, 2, 4] 129.03461596915875
[0, 3, 2, 5] 118.83117770896315
[1, 2, 3, 4] 56.10989169721264
[1, 2, 3, 5] 12.750734071856304
[1, 3, 2, 4] 56.10989169721264
[1, 3, 2, 5] 12.750734071855733
[2, 4, 3, 5] 56.3856286987971
[2, 5, 3, 4] 56.3856286987971


Even though each candidate avoids double-counting within itself, the same combination of four indexes can be found among the candidates. We want only one of each.

Let's collect these Higgs candidates by unique sets of indexes. The `sorted` function sorts a list, and `tuple` makes it possible to use them as keys in a dict.

In [10]:
higgs_candidates_step2 = {}

for higgs_candidate in higgs_candidates_step1:
    combination = tuple(sorted(higgs_candidate["index"]))

    if combination not in higgs_candidates_step2:
        higgs_candidates_step2[combination] = []

    higgs_candidates_step2[combination].append(higgs_candidate)

This `higgs_candidates_step2` has deep structure:

  * Keys are sets combinations of lepton indexes, without regard for their original order.
  * Values are a list of decay trees.
    - Each element of that list has a candidate Higgs mass and two candidate Z masses.

In [11]:
for combination in higgs_candidates_step2:
    print(combination)
    for higgs_candidate in higgs_candidates_step2[combination]:
        z_boson1, z_boson2 = higgs_candidate["Z bosons"]
        print(
            "    Higgs:",
            higgs_candidate["H boson"].mass,
            "Z:",
            z_boson1.mass,
            z_boson2.mass,
        )

(0, 1, 2, 3)
    Higgs: 118.25980390632363 Z: 94.65200565609616 3.087732909498452
    Higgs: 118.25980390632363 Z: 62.03397488944119 3.4170504361020386
(0, 2, 3, 4)
    Higgs: 129.03461596915875 Z: 94.65200565609616 26.45024522236556
    Higgs: 129.03461596915875 Z: 62.03397488944119 45.69023328291948
(0, 2, 3, 5)
    Higgs: 118.83117770896321 Z: 94.65200565609616 3.27373703909123
    Higgs: 118.83117770896315 Z: 62.03397488944119 3.6622583780140623
(1, 2, 3, 4)
    Higgs: 56.10989169721264 Z: 3.4170504361020386 26.45024522236556
    Higgs: 56.10989169721264 Z: 3.087732909498452 45.69023328291948
(1, 2, 3, 5)
    Higgs: 12.750734071856304 Z: 3.4170504361020386 3.27373703909123
    Higgs: 12.750734071855733 Z: 3.087732909498452 3.6622583780140623
(2, 3, 4, 5)
    Higgs: 56.3856286987971 Z: 45.69023328291948 3.27373703909123
    Higgs: 56.3856286987971 Z: 3.6622583780140623 26.45024522236556


<br><br><br><br><br>

## Exercise part 3

One of the selections that the 2012 Higgs discovery analysis applied was:

  * 12 GeV/$c^2$ < smallest Z mass < 120 GeV/$c^2$
  * 40 GeV/$c^2$ < largest Z mass < 120 GeV/$c^2$

because this is expected of real Higgs decays.

Apply the Z mass constraint to these Higgs candidates.

In [12]:
higgs_candidates_step3 = {}

for combination in higgs_candidates_step2:
    higgs_candidates_step3[combination] = []

    for higgs_candidate in higgs_candidates_step2[combination]:
        z_boson1, z_boson2 = higgs_candidate["Z bosons"]
        if z_boson1.mass < z_boson2.mass:
            smallest_z_mass, largest_z_mass = z_boson1.mass, z_boson2.mass
        else:
            smallest_z_mass, largest_z_mass = z_boson2.mass, z_boson1.mass

        if 12 < smallest_z_mass < 120 and 40 < largest_z_mass < 120:
            higgs_candidates_step3[combination].append(higgs_candidate)

You could have used

```python
smallest_z_mass = z_boson1.mass if z_boson1.mass < z_boson2.mass else z_boson2.mass
largest_z_mass = z_boson1.mass if z_boson1.mass >= z_boson2.mass else z_boson2.mass
```

or

```python
smallest_z_mass, largest_z_mass = sorted([z_boson1.mass, z_boson2.mass])
```

or several other, equivalent, variants.

In the end,

In [13]:
for combination in higgs_candidates_step3:
    print(combination)
    for higgs_candidate in higgs_candidates_step3[combination]:
        z_boson1, z_boson2 = higgs_candidate["Z bosons"]
        print(
            "    Higgs:",
            higgs_candidate["H boson"].mass,
            "Z:",
            z_boson1.mass,
            z_boson2.mass,
        )

(0, 1, 2, 3)
(0, 2, 3, 4)
    Higgs: 129.03461596915875 Z: 94.65200565609616 26.45024522236556
    Higgs: 129.03461596915875 Z: 62.03397488944119 45.69023328291948
(0, 2, 3, 5)
(1, 2, 3, 4)
(1, 2, 3, 5)
(2, 3, 4, 5)
